#### Use this for sentiment analysis with Logistic Regression and Naive Bayes
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_Sentiment_Analysis.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_KNN_RecommenderSystem.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Naive%20Bayes/Amazon%20reviews_Naive%20Bayes.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Recommender%20System/Recommender%20System.ipynb

Removing Contractions - https://medium.com/analytics-vidhya/sentiment-analysis-on-amazon-reviews-using-tf-idf-approach-c5ab4c36e7a1


http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook_WIP.ipynb
http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Recommendation%2BSystem%2BNotebook.ipynb

http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/Akash_collaborative_user_base..ipynb

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [2]:
import re

In [3]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv("sample30.csv")

In [5]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
print(df.columns)
print(df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


In [7]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [8]:
#Tokenize the review text to words
reg = re.compile('[^a-z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

In [9]:
df['reviews_tokenized'] = df['reviews_text'].apply(tokenizeReview)

In [10]:
df.shape

(30000, 16)

In [11]:
df = df.drop(['reviews_text'], axis=1)

In [12]:
df.shape

(30000, 15)

In [13]:
df['reviews_tokenized'].head(5)

0    i love this album it s very good more to the h...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

In [14]:
df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [15]:
def senti(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = df['reviews_tokenized']
y = df['user_sentiment'].apply(senti)

In [16]:
print(len(X))
print(len(y))

30000
30000


In [17]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

In [18]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

In [19]:
def model_fit(X, y, feature_model,ml_model,coef_show=1):
    
    X_features = feature_model.fit_transform(X)
    print('# features: {}'.format(X_features.shape[1]))
    
#    tempX = feature_model.transform(['this is an awful product', 'i am satisfied with this. will come back again'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size = 0.2, random_state=43)
    clf = ml_model.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    model_performance = classification_report(y_test, clf_pred)
    print ('accuracy of the model: ', accuracy)
    print('')
    print(model_performance)

#    print(X_train)
#    print("---")
#    print(X_test)
#    print("---")
#    print(dir(ml_model))
    
#    tempPred = ml_model.predict(tempX)
#    print('---Prediction----')
#    print(tempPred)
    
#    print(vars(X_test))
#    print("indices - ", len(X_test.indices))
#    print("indptr - ", len(X_test.indptr))
#    print("data - ", len(X_test.data))
    
#    print(dir(X_test))
#    print("X_ARRAY ", X_test._get_arrayXarray(0, 529))
    
    
    if coef_show == 1: 
        w = feature_model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('Top 10 positive features (variables)')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('Top 10 negative features (variables)')        
        print(coeff_df.tail(20).to_string(index=False))
       
    return X_test, clf_pred

xtest, pred = model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    

In [20]:
xtest, pred = model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    

In [21]:
print(xtest)
print(pred) 

  (0, 529)	0.22241595149271842
  (0, 47)	0.20964319801245335
  (0, 720)	0.696980923623149
  (0, 771)	0.21593130586186834
  (0, 90)	0.40026789622812714
  (0, 969)	0.1885671404031856
  (0, 371)	0.16567427089889267
  (0, 511)	0.1772086562204112
  (0, 940)	0.1333247534125354
  (0, 619)	0.20415080720516499
  (0, 61)	0.11974647057071268
  (0, 343)	0.13457303762618536
  (0, 832)	0.16627431734861037
  (1, 569)	0.24218436278559
  (1, 154)	0.6098511233084832
  (1, 526)	0.4614886748852459
  (1, 619)	0.5597691993889182
  (1, 524)	0.20764148956715778
  (2, 902)	0.6130275606318966
  (2, 859)	0.6486652086683559
  (2, 677)	0.22152244726222683
  (2, 147)	0.22152244726222683
  (2, 718)	0.22001360570007655
  (2, 524)	0.23849587206847778
  (3, 110)	0.5440276702573296
  :	:
  (5998, 3)	0.2557293834011859
  (5998, 110)	0.32355073600328416
  (5998, 570)	0.23649604270272198
  (5998, 283)	0.36865793862816465
  (5998, 752)	0.3047600169042265
  (5998, 940)	0.23511843058393603
  (5998, 155)	0.28812486505187546
  